<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/boosting/Credit_Scorecards_with_XGBoost_and_W&B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{xgboost_vehicle_loan_default} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" /><br>

<!--- @wandbcode{xgboost_vehicle_loan_default} -->

<img src="http://wandb.me/mini-diagram" width="600" alt="Weights & Biases" />

# In this notebook

In this colab we'll cover how Weights and Biases enables regulated entities to
- **Track and version** their data ETL pipelines (locally or in cloud services such as S3 and GCS)
- **Track experiment results** and store trained models
- **Visually inspect** multiple evaluation metrics
- **Optimize performance** with hyperparameter sweeps

# Setup

In [1]:
import wandb
from pathlib import Path
import numpy as np
import sys
import os
import pandas as pd
from sklearn import model_selection
import xgboost as xgb


## Login to W&B
Login to Weights and Biases

In [2]:
import wandb

WANDB_PROJECT ='titanic_survived'
os.environ['WANDB_NOTEBOOK_NAME'] = 'Titanic_Survived_with_XGBoost.ipynb'

In [3]:
wandb.login()

wandb: Currently logged in as: max-deep. Use `wandb login --relogin` to force relogin


True

#### Upload Data with W&B Artifacts

In [6]:
run = wandb.init(project=WANDB_PROJECT, job_type='add-dataset')

Download the subset of the vehicle loan default data from W&B, this contains `train.csv` and `val.csv` files as well as some utils files.

In [7]:
artifact = wandb.Artifact(name="titanic-dataset", type="dataset")
artifact.add_dir(local_path="./titanic")  # Add dataset directory to artifact
run.log_artifact(artifact)

wandb: Adding directory to artifact (.\titanic)... Done. 0.1s


<Artifact titanic-dataset>

In [8]:
run.finish()

## Get Train/Validation Split

Here we show an alternative pattern for how to create a wandb run object. In the cell below, the code to split the dataset is wrapped with a call to `wandb.init() as run`.

Here we will:

- Start a wandb run
- Download our one-hot-encoded dataset from Artifacts
- Do the Train/Val split and log the params used in the split
- Log the new `trndat` and `valdat` datasets to Artifacts
- Finish the wandb run automatically

In [12]:
target = 'Survived'

In [14]:
with wandb.init(project=WANDB_PROJECT, job_type='train-val-split') as run:     # config is optional here

    # Download the subset of the vehicle loan default data from W&B
    dataset_art = run.use_artifact('max-deep/titanic_survived/titanic-dataset:v0', type='dataset')
    root_dir = Path(sys.path[0])  # get this notebook path
    data_dir = root_dir / 'dataset'
    data_dir.mkdir(exist_ok=True)
    
    dataset_dir = dataset_art.download(data_dir)
    dataset = pd.read_csv(f'{data_dir}/train.csv')

    # Set Split Params
    test_size = 0.20
    random_state = 42

    # Log the splilt params
    run.config.update({'test_size':test_size, 'random_state': random_state})

    # Do the Train/Val Split
    trndat, valdat = model_selection.train_test_split(dataset, test_size=test_size,
                                                      random_state=random_state, stratify=dataset[[target]])

    print(f'Train dataset size: {trndat[target].value_counts()} \n')
    print(f'Validation dataset sizeL {valdat[target].value_counts()}')

    # Save split datasets
    data_split_dir = root_dir / 'split-dataset'
    data_split_dir.mkdir(exist_ok=True)
    train_path = data_split_dir/'train.csv'
    val_path = data_split_dir/'val.csv'
    trndat.to_csv(train_path, index=False)
    valdat.to_csv(val_path, index=False)

    # Create a new artifact for the processed data, including the function that created it, to Artifacts
    split_ds_art = wandb.Artifact(name='titanic-dataset-split',
                                        type='train-val-dataset',
                                        description='Processed dataset split into training and validation sets',
                                        metadata={'test_size': test_size, 'random_state': random_state}
                                     )

    # Attach our processed data to the Artifact
    split_ds_art.add_file(train_path)
    split_ds_art.add_file(val_path)

    # Log the Artifact
    run.log_artifact(split_ds_art)

wandb:   3 of 3 files downloaded.  


Train dataset size: Survived
0    439
1    273
Name: count, dtype: int64 

Validation dataset sizeL Survived
0    110
1     69
Name: count, dtype: int64


#### Inspect Training Dataset
Get an overview of the training dataset

In [3]:
# Read our CSV into a new DataFrame
train_df = pd.read_csv("dataset/train.csv")
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
# Counting NaN values in all columns
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
train_df['Survived'].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

### Log Dataset with W&B Tables

With W&B Tables you can log, query, and analyze tabular data that contains rich media such as images, video, audio and more. With it you can understand your datasets, visualize model predictions, and share insights, for more see more in our [W&B Tables Guide](https://docs.wandb.ai/guides/data-vis)

In [10]:
# Create a wandb run, with an optional "log-dataset" job type to keep things tidy
run = wandb.init(project=WANDB_PROJECT, job_type='log-dataset')  # config is optional here

# Create a W&B Table and log 1000 random rows of the dataset to explore
table = wandb.Table(dataframe=train_df.sample(100))

# Log the Table to your W&B workspace
wandb.log({'sample-dataset': table})

# Close the wandb run
wandb.finish()

# Modelling

## Fit the XGBoost Model

We will now fit an XGBoost model to classify whether a vehicle loan application will result in a default or not

### Training on GPU
If you'd like to train your XGBoost model on your GPU, simply change set the following in the parameters you pass to XGBoost:

```
'tree_method': 'gpu_hist'
```

#### 1) Initialise a W&B Run

In [4]:
run = wandb.init(project=WANDB_PROJECT, job_type='train-model')

#### 2) Setup and Log the Model Parameters

In [5]:
params = {'max_depth': 6 ## default: 6
  , 'min_child_weight': 1  ## default: 1
  , 'subsample': 1 ## default: 1
  , 'colsample_bytree': 1 ## default: 1
  , 'n_estimators': 100 ## default: 100
  , 'learning_rate': 0.3 ## default: 0.3
  , 'early_stopping_rounds': None ## optional
  , 'random_state': 42
  , 'tree_method': 'hist'
  , 'device': 'cuda' 
    }

Log the xgboost training parameters to the W&B run config

#### 3) Load the Training Data from W&B Artifacts

In [6]:
def load_training_data(run, data_dir, artifact_name:str, train_file='train.csv', val_file='val.csv'):
    data_art = run.use_artifact(artifact_name)
    dataset_dir = data_art.download(data_dir)
    df_train = pd.read_csv(dataset_dir + train_file)
    df_val = pd.read_csv(dataset_dir + val_file)
    return df_train, df_val

In [7]:
# Load our training data from Artifacts
df_train, df_val= load_training_data(run=run, data_dir='split-dataset/',
                                    artifact_name='titanic-dataset-split:latest')

wandb:   2 of 2 files downloaded.  


In [8]:
# Feature selection to drop irrelevant columns
drop_list = ['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'] 

# Split data into target and feature variables

# Train
y_train = df_train['Survived']
X_train = df_train.drop(columns=drop_list)

# Val
y_val = df_val['Survived']
X_val = df_val.drop(columns=drop_list)

#### 4) Fit the model, log results to W&B and save model to W&B Artifacts

To log all our xgboost model parameters we used the `WandbCallback`. This will . See the [W&B docs](https://docs.wandb.ai/guides/integrations), including documentation for other libraries that have integrated W&B including LightGBM and more.

In [9]:
from wandb.xgboost import WandbCallback
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [10]:
# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[("one_hot", OneHotEncoder(), selector(dtype_include="object"))], # One Hot Encoding
    remainder="passthrough",  # Leave numerical variables unchanged
)

# Create pipeline
pipeline = Pipeline([("preprocessor", preprocessor), # Preprocessing
                     ("classifier", xgb.XGBClassifier(**params,
                             callbacks=[WandbCallback(log_model=True)])) # XGBoost classifier
                     ])

pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot', OneHotEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B53FCC6080>)])),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None,
                               callbacks=[<wandb.integration.xgboost.xgboost.WandbCallback object at 0x000001B51E346D40>],
                               c...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.3,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None, min_child_weight=1,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               random_state=42, ...))])

In [11]:
# Perform cross-validation
cv_scores = model_selection.cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

c:\Users\Maxpo\Documents\GitHub\tabular-data-prediction-pipeline\.venv\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
c:\Users\Maxpo\Documents\GitHub\tabular-data-prediction-pipeline\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [12:12:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
c:\Users\Maxpo\Documents\Git

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot', OneHotEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B53FCC6080>)])),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None,
                               callbacks=[<wandb.integration.xgboost.xgboost.WandbCallback object at 0x000001B51E346D40>],
                               c...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.3,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None, min_child_weight=1,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               random_state=42, ...))])

#### 5) Log Additional Train and Evaluation Metrics to W&B

In [12]:
from scipy.stats import ks_2samp

In [13]:
bstr = pipeline.named_steps['classifier'].get_booster()

# Get train and validation predictions
y_train_preds = pipeline.predict_proba(X_train)[:,1]
y_val_preds = pipeline.predict_proba(X_val)[:,1]

# Log additional Train metrics
false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_train, y_train_preds)
run.summary['train_ks_stat'] = max(true_positive_rate - false_positive_rate)
run.summary['train_auc'] = metrics.auc(false_positive_rate, true_positive_rate)
run.summary['train_log_loss'] = -(y_train * np.log(y_train_preds) + (1-y_train) * np.log(1-y_train_preds)).sum() / len(y_train)
run.summary['mean_cv_accuracy'] =  np.mean(cv_scores)

# Log additional Validation metrics
fpr, tpr, thresholds = metrics.roc_curve(y_val, y_val_preds)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

y_val_preds_binary = np.where(y_val_preds >= optimal_threshold, 1, 0)

ks_stat, ks_pval = ks_2samp(y_val_preds[y_val==1], y_val_preds[y_val==0])
run.summary["val_ks_2samp"] = ks_stat
run.summary["val_ks_pval"] = ks_pval
run.summary["val_auc"] = metrics.roc_auc_score(y_val, y_val_preds)
run.summary["val_acc_0.5"] = metrics.accuracy_score(y_val, np.where(y_val_preds >= 0.5, 1, 0))
run.summary[f"val_acc_best_trh"] = metrics.accuracy_score(y_val, y_val_preds_binary)
run.summary["val_log_loss"] = -(y_val * np.log(y_val_preds)
                                     + (1-y_val) * np.log(1-y_val_preds)).sum() / len(y_val)


#### 6) Log the ROC Curve To W&B

In [14]:
# Log the ROC curve to W&B
valYpreds_2d = np.array([1-y_val_preds, y_val_preds])  # W&B expects a 2d array
y_val_arr = y_val.values
d = 0
while len(valYpreds_2d.T) > 10000:
    d +=1
    valYpreds_2d = valYpreds_2d[::1, ::d]
    y_val_arr = y_val_arr[::d]

run.log({"ROC_Curve" : wandb.plot.roc_curve(y_val_arr, valYpreds_2d.T,
                                           labels=['Dead','Survived'],
                                           )})

In [15]:
wandb.log({"Confusion_Matrix" :wandb.sklearn.plot_confusion_matrix(y_val, 
                                                                   y_val_preds_binary, ['Dead','Survived'])})


#### Finish the W&B Run

In [16]:
run.finish()

epoch,▁▂▃▄▅▇█▂▃▄▅▆▇█▂▃▄▅▇█▁▂▃▅▆▇█▂▃▄▅▆▇▁▂▃▅▆▇█
epoch,99
mean_cv_accuracy,0.80066
train_auc,0.99705
train_ks_stat,0.95435
train_log_loss,0.10546
val_acc_0.5,0.81006
val_acc_best_trh,0.82123
val_auc,0.82543
val_ks_2samp,0.60646
val_ks_pval,0.0


In [17]:
df_test = pd.read_csv('dataset/test.csv')

Now that we've trained a single model, lets try and optimize its performance by running a Hyperparameter Sweep.

# HyperParameter Sweep

Weights and Biases also enables you to do hyperparameter sweeps, either with our own [Sweeps functionality](https://docs.wandb.ai/guides/sweeps/python-api) or with our [Ray Tune integration](https://docs.wandb.ai/guides/sweeps/advanced-sweeps/ray-tune). See [our docs](https://docs.wandb.ai/guides/sweeps/python-api) for a full guide of how to use more advanced hyperparameter sweeps options.

#### Define the Sweep Config
First we define the hyperparameters to sweep over as well as the type of sweep to use, we'll do a bayes search over the most common hyperparameters!

In [18]:
sweep_configuration = {
  'method': 'bayes',
  'name': 'sweep',
  'metric': {
        'goal': 'maximize', 
        'name': 'val_acc_best_trh'
        },
  "parameters" : {
    "max_depth" :{
      "min": 3,
      "max": 10
    },
    "min_child_weight" :{
      "min": 1,
      "max": 10
    },
    "subsample" :{
      "min": 0.5,
      "max": 1.0
    },
    "colsample_bytree" :{
      "min": 0.5,
      "max": 1.0
    },
    "n_estimators":{
      'values': [30, 60, 90, 120, 150, 180, 210]
    },
    "learning_rate" :{
      "min": 0.001,
      "max": 1.0
    },
  }
}

sweep_id = wandb.sweep(sweep_configuration, project=WANDB_PROJECT)

Create sweep with ID: 6zycoh6c
Sweep URL: https://wandb.ai/max-deep/titanic_survived/sweeps/6zycoh6c


#### Define the Training Function

Then we define the function that will train our model using these hyperparameters. Note that `job_type='sweep'` when initialising the run, so that we can easily filter out these runs from our main workspace if we need to

In [19]:
def train():
    with wandb.init(job_type="sweep") as run:

        params = {'max_depth': run.config['max_depth'] ## default: 6
            , 'min_child_weight': run.config['min_child_weight']  ## default: 1
            , 'subsample': run.config['subsample'] ## default: 1
            , 'colsample_bytree': run.config['colsample_bytree'] ## default: 1
            , 'n_estimators': run.config['n_estimators'] ## default: 100
            , 'learning_rate': run.config['learning_rate'] ## default: 0.3
            , 'early_stopping_rounds': None ## optional
            , 'random_state': 42
            , 'tree_method': 'hist'
            , 'device': 'cuda' 
        }
        
        # Create pipeline
        pipeline = Pipeline([("preprocessor", preprocessor), # Preprocessing
                            ("classifier", xgb.XGBClassifier(**params,
                                    callbacks=[WandbCallback(log_model=True)])) # XGBoost classifier
                            ])
        
        # Perform cross-validation
        cv_scores = model_selection.cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')

        # Fit the pipeline on the training data
        pipeline.fit(X_train, y_train)

        bstr = pipeline.named_steps['classifier'].get_booster()

        # Get train and validation predictions
        y_train_preds = pipeline.predict_proba(X_train)[:,1]
        y_val_preds = pipeline.predict_proba(X_val)[:,1]

        # Log additional Train metrics
        false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_train, y_train_preds)
        run.summary['train_ks_stat'] = max(true_positive_rate - false_positive_rate)
        run.summary['train_auc'] = metrics.auc(false_positive_rate, true_positive_rate)
        run.summary['train_log_loss'] = -(y_train * np.log(y_train_preds) + (1-y_train) * np.log(1-y_train_preds)).sum() / len(y_train)
        run.summary['mean_cv_accuracy'] =  np.mean(cv_scores)

        # Log additional Validation metrics
        fpr, tpr, thresholds = metrics.roc_curve(y_val, y_val_preds)
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]

        y_val_preds_binary = np.where(y_val_preds >= optimal_threshold, 1, 0)

        ks_stat, ks_pval = ks_2samp(y_val_preds[y_val==1], y_val_preds[y_val==0])
        run.summary["val_ks_2samp"] = ks_stat
        run.summary["val_ks_pval"] = ks_pval
        run.summary["val_auc"] = metrics.roc_auc_score(y_val, y_val_preds)
        run.summary["val_acc_0.5"] = metrics.accuracy_score(y_val, np.where(y_val_preds >= 0.5, 1, 0))
        run.summary[f"val_acc_best_trh"] = metrics.accuracy_score(y_val, y_val_preds_binary)
        run.summary["val_log_loss"] = -(y_val * np.log(y_val_preds)
                                            + (1-y_val) * np.log(1-y_val_preds)).sum() / len(y_val)

        


#### Run the Sweeps Agent

In [20]:
count = 100 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

wandb: Agent Starting Run: vb8bglsu with config:
wandb: 	colsample_bytree: 0.6273986472917432
wandb: 	learning_rate: 0.04005619921185925
wandb: 	max_depth: 3
wandb: 	min_child_weight: 4
wandb: 	n_estimators: 150
wandb: 	subsample: 0.8983677642301183


epoch,▁▂▃▄▅▇█▁▃▄▅▆▇▁▂▃▄▅▆█▁▂▃▅▆▇█▂▃▄▅▆▇▁▂▃▄▆▇█
epoch,149
mean_cv_accuracy,0.81471
train_auc,0.9109
train_ks_stat,0.69652
train_log_loss,0.36107
val_acc_0.5,0.77095
val_acc_best_trh,0.81564
val_auc,0.83735
val_ks_2samp,0.62978
val_ks_pval,0.0


wandb: Agent Starting Run: 5ls3etez with config:
wandb: 	colsample_bytree: 0.7093466138800043
wandb: 	learning_rate: 0.5726972199509932
wandb: 	max_depth: 7
wandb: 	min_child_weight: 8
wandb: 	n_estimators: 90
wandb: 	subsample: 0.9630017883356852


epoch,▁▂▃▅▅▇█▁▃▄▅▆▇▁▂▃▄▅▆█▁▂▄▄▅▇█▁▃▄▅▆▇█▂▃▄▅▆█
epoch,89
mean_cv_accuracy,0.82592
train_auc,0.94986
train_ks_stat,0.77742
train_log_loss,0.28161
val_acc_0.5,0.82123
val_acc_best_trh,0.8324
val_auc,0.84177
val_ks_2samp,0.61383
val_ks_pval,0.0


wandb: Agent Starting Run: 094s90cm with config:
wandb: 	colsample_bytree: 0.5475462743343226
wandb: 	learning_rate: 0.29043571667125223
wandb: 	max_depth: 3
wandb: 	min_child_weight: 8
wandb: 	n_estimators: 210
wandb: 	subsample: 0.9844999263411224


...

epoch,▁▂▃▄▅▇▇▁▃▃▅▆▇█▁▃▄▅▇▇▁▂▃▅▅▇█▁▃▄▅▆▇▁▂▃▅▅▇█
epoch,59
mean_cv_accuracy,0.80487
train_auc,0.93155
train_ks_stat,0.74445
train_log_loss,0.32133
val_acc_0.5,0.81006
val_acc_best_trh,0.83799
val_auc,0.85296
val_ks_2samp,0.60672
val_ks_pval,0.0


wandb: Agent Starting Run: 2ipu4lo2 with config:
wandb: 	colsample_bytree: 0.8868652081810273
wandb: 	learning_rate: 0.665816808280413
wandb: 	max_depth: 10
wandb: 	min_child_weight: 9
wandb: 	n_estimators: 60
wandb: 	subsample: 0.7881637502413787


epoch,▁▂▃▄▅▇▇▁▃▃▅▆▇█▁▃▄▅▇▇▁▂▃▅▅▇█▁▃▄▅▆▇▁▂▃▅▅▇█
epoch,59
mean_cv_accuracy,0.8133
train_auc,0.9442
train_ks_stat,0.7788
train_log_loss,0.29113
val_acc_0.5,0.82682
val_acc_best_trh,0.8324
val_auc,0.8552
val_ks_2samp,0.62464
val_ks_pval,0.0


wandb: Agent Starting Run: x9d43tnr with config:
wandb: 	colsample_bytree: 0.9372488599535576
wandb: 	learning_rate: 0.7615852672710105
wandb: 	max_depth: 10
wandb: 	min_child_weight: 10
wandb: 	n_estimators: 150
wandb: 	subsample: 0.7598445841841153


epoch,▁▂▃▄▅▇█▁▃▄▅▆▇▁▂▃▄▅▆█▁▂▃▅▆▇█▂▃▄▅▆▇▁▂▃▄▆▇█
epoch,149
mean_cv_accuracy,0.82175
train_auc,0.95871
train_ks_stat,0.79515
train_log_loss,0.25505
val_acc_0.5,0.82123
val_acc_best_trh,0.82123
val_auc,0.84657
val_ks_2samp,0.60105
val_ks_pval,0.0


In [21]:
run.finish()

## W&B already in your favorite ML library

Weights and Biases has integrations in all of your favourite ML and Deep Learning libraries such as:

- Pytorch Lightning
- Keras
- Hugging Face
- JAX
- Fastai
- XGBoost
- Sci-Kit Learn
- LightGBM

**See [W&B integrations for details](https://docs.wandb.ai/guides/integrations)**